#Parser   

```
import requests
import pandas as pd
from bs4 import BeautifulSoup


url = 'https://code.s3.yandex.net/learning-materials/data-analyst/festival_news/index.html'
req = requests.get(url)
soup = BeautifulSoup(req.text, 'lxml') 
table = soup.find('table', attrs={'id': 'best_festivals'})
heading_table = []
for row in table.find_all('th'):
        heading_table.append(row.text)
content=[]
for row in table.find_all('tr'): 
    if not row.find_all('th'): 
            content.append([element.text for element in row.find_all('td')])
festivals = pd.DataFrame(content, columns=heading_table)
print(festivals)


```

#Работа с базой  


Менеджер авиаперевозчика готовит исследование. Он хочет выявить связь между числом полётов, типом самолётов и календарём музыкальных фестивалей. Помогите ему получить необходимые данные.  

Описание данных  




**Таблица airports — информация об аэропортах:**  


- `airport_code` — трёхбуквенный код аэропорта  
- `airport_name` — название аэропорта  
- `city` — город  
- `timezone` — временная зона  


**Таблица aircrafts — информация о самолётах:**  


- `aircraft_code` — код модели самолёта  
- `model` — модель самолёта  
- `range` — дальность полёта  


**Таблица tickets — информация о билетах:**   


- `ticket_no` — уникальный номер билета  
- `passenger_id` — персональный идентификатор пассажира  
- `passenger_name` — имя и фамилия пассажира  


**Таблица flights — информация о рейсах**   


- `flight_id` — уникальный идентификатор рейса  
- `departure_airport` — аэропорт вылета  
- `departure_time` — дата и время вылета  
- `arrival_airport` — аэропорт прилёта  
- `arrival_time` — дата и время прилёта  
- `aircraft_code` — id самолёта  


**Таблица ticket_flights — стыковая таблица «рейсы-билеты»**  

- `ticket_no` — номер билета  
- `flight_id` — идентификатор рейса  
- `Таблица festivals` — информация о фестивалях  
- `festival_id` — уникальный номер фестиваля   
- `festival_date` — дата проведения фестиваля  
- `festival_city` — город проведения фестиваля  
- `festival_name` — название фестиваля  


# Схема таблиц

![](https://i.ibb.co/z74gkxK/PK-FK-weather-1-1585141477.jpg)

### 1 запрос


Изучите таблицу `airports` и выведите список городов (`city`), в которых есть аэропорты.

```
SELECT DISTINCT t.city
FROM   airports t 
```

### 2 запрос



Изучите таблицу `flights` и подсчитайте количество вылетов (`flight_id`) из каждого аэропорта вылета (`departure_airport`). Назовите переменную cnt_flights и выведите её вместе со столбцом `departure_airport` — сначала `departure_airport`, потом `cnt_flights`. Результат отсортируйте в порядке убывания количества вылетов.

```
SELECT t.departure_airport,
       Count(flight_id) AS cnt_flights
FROM   flights t
GROUP  BY t.departure_airport
ORDER  BY Count(flight_id) DESC 

```

### 3 запрос



Найдите количество рейсов на каждой модели самолёта с вылетом в сентябре 2018 года. Назовите получившееся поле `flights_amount` и выведите его. Также напечатайте на экране поле model. Столбцы в таблице должны быть выведены в следующем порядке

<font face="Courier New" size="2">
<font color = "royalblue">SELECT</font>&nbsp;<font color = "cyan">a</font><font color = "silver">.</font><font color = "cyan">model</font>&nbsp;&nbsp;<font color = "royalblue">AS</font>&nbsp;<font color = "cyan">model</font><font color = "silver">,</font>
<br/>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<font color = "#FF0080"><b>Count</b></font><font color = "cyan">(</font><font color = "silver">*</font><font color = "cyan">)</font>&nbsp;<font color = "royalblue">AS</font>&nbsp;<font color = "cyan">flights_amount</font>
<br/><font color = "royalblue">FROM</font>&nbsp;&nbsp;&nbsp;<font color = "cyan">flights</font>&nbsp;<font color = "cyan">f</font>
<br/>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<font color = "cyan">inner</font>&nbsp;<font color = "cyan">join</font>&nbsp;<font color = "cyan">aircrafts</font>&nbsp;<font color = "cyan">a</font>
<br/>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<font color = "royalblue">ON</font>&nbsp;<font color = "cyan">a</font><font color = "silver">.</font><font color = "cyan">aircraft_code</font>&nbsp;<font color = "silver">=</font>&nbsp;<font color = "cyan">f</font><font color = "silver">.</font><font color = "cyan">aircraft_code</font>
<br/><font color = "royalblue">WHERE</font>&nbsp;&nbsp;<font color = "#FF0080"><b>Cast</b></font><font color = "cyan">(</font><font color = "cyan">f</font><font color = "silver">.</font><font color = "cyan">departure_time</font>&nbsp;<font color = "royalblue">AS</font>&nbsp;<font color = "black"><i>DATE</i></font><font color = "cyan">)</font>&nbsp;<font color = "royalblue">BETWEEN</font>&nbsp;<font color = "green">'2018-09-01'</font>&nbsp;<font color = "royalblue">AND</font>&nbsp;<font color = "green">'2018-09-30'</font>
<br/><font color = "royalblue">GROUP</font>&nbsp;&nbsp;<font color = "royalblue">BY</font>&nbsp;<font color = "cyan">a</font><font color = "silver">.</font><font color = "cyan">model</font>&nbsp;
</font>


### 4 запрос



Посчитайте отправленные в сентябре рейсы самолётов следующих моделей:  


Boeing,  
Airbus,  
другие (“other”). 

<font face="Courier New" size="2">
<font color = "royalblue">WITH</font>&nbsp;<font color = "cyan">SUBQ</font>
<br/>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<font color = "royalblue">AS</font>&nbsp;<font color = "cyan">(</font><font color = "royalblue">SELECT</font>&nbsp;<font color = "cyan">a</font><font color = "silver">.</font><font color = "cyan">AIRCRAFT_CODE</font><font color = "silver">,</font>
<br/>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<font color = "royalblue">CASE</font>
<br/>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<font color = "royalblue">WHEN</font>&nbsp;<font color = "cyan">(</font>&nbsp;<font color = "cyan">a</font><font color = "silver">.</font><font color = "cyan">MODEL</font>&nbsp;<font color = "royalblue">LIKE</font>&nbsp;<font color = "green">'Boeing%'</font>&nbsp;<font color = "cyan">)</font>&nbsp;<font color = "royalblue">THEN</font>&nbsp;<font color = "green">'Boeing'</font>
<br/>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<font color = "royalblue">WHEN</font>&nbsp;<font color = "cyan">(</font>&nbsp;<font color = "cyan">a</font><font color = "silver">.</font><font color = "cyan">MODEL</font>&nbsp;<font color = "royalblue">LIKE</font>&nbsp;<font color = "green">'Airbus%'</font>&nbsp;<font color = "cyan">)</font>&nbsp;<font color = "royalblue">THEN</font>&nbsp;<font color = "green">'Airbus'</font>
<br/>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<font color = "royalblue">ELSE</font>&nbsp;<font color = "green">'other'</font>
<br/>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<font color = "royalblue">END</font>&nbsp;<font color = "royalblue">AS</font>&nbsp;<font color = "cyan">model</font>
<br/>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<font color = "royalblue">FROM</font>&nbsp;&nbsp;&nbsp;<font color = "cyan">aircrafts</font>&nbsp;<font color = "cyan">a</font><font color = "cyan">)</font>
<br/><font color = "royalblue">SELECT</font>&nbsp;<font color = "cyan">a</font><font color = "silver">.</font><font color = "cyan">MODEL</font><font color = "silver">,</font>
<br/>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<font color = "#FF0080"><b>COUNT</b></font><font color = "cyan">(</font><font color = "silver">*</font><font color = "cyan">)</font>
<br/><font color = "royalblue">FROM</font>&nbsp;&nbsp;&nbsp;<font color = "cyan">flights</font>&nbsp;<font color = "cyan">f</font>
<br/>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<font color = "cyan">JOIN</font>&nbsp;<font color = "cyan">SUBQ</font>&nbsp;<font color = "cyan">a</font>
<br/>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<font color = "royalblue">ON</font>&nbsp;<font color = "cyan">a</font><font color = "silver">.</font><font color = "cyan">AIRCRAFT_CODE</font>&nbsp;<font color = "silver">=</font>&nbsp;<font color = "cyan">f</font><font color = "silver">.</font><font color = "cyan">AIRCRAFT_CODE</font>
<br/><font color = "royalblue">WHERE</font>&nbsp;&nbsp;<font color = "cyan">f</font><font color = "silver">.</font><font color = "cyan">DEPARTURE_TIME</font>&nbsp;<font color = "silver">::</font>&nbsp;<font color = "royalblue">DATE</font>&nbsp;<font color = "royalblue">BETWEEN</font>&nbsp;<font color = "green">'2018-09-01'</font>&nbsp;<font color = "royalblue">AND</font>&nbsp;<font color = "green">'2018-09-30'</font>
<br/><font color = "royalblue">GROUP</font>&nbsp;&nbsp;<font color = "royalblue">BY</font>&nbsp;<font color = "cyan">a</font><font color = "silver">.</font><font color = "cyan">MODEL</font>&nbsp;
</font>


### 5 запрос




Посчитайте среднее количество прибывающих рейсов в день для каждого города за август 2018 года. Назовите получившееся поле `average_flights`, вместе с ним выведите столбец `city`.

<font face="Courier New" size="2">
<font color = "royalblue">WITH</font>&nbsp;<font color = "cyan">FLIGHTS_08</font>
<br/>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<font color = "royalblue">AS</font>&nbsp;<font color = "cyan">(</font><font color = "royalblue">SELECT</font>&nbsp;<font color = "cyan">a</font><font color = "silver">.</font><font color = "cyan">CITY</font>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<font color = "royalblue">AS</font>&nbsp;<font color = "cyan">city</font><font color = "silver">,</font>
<br/>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<font color = "#FF0080"><b>COUNT</b></font><font color = "cyan">(</font><font color = "cyan">f</font><font color = "silver">.</font><font color = "cyan">FLIGHT_ID</font><font color = "cyan">)</font>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<font color = "royalblue">AS</font>&nbsp;<font color = "cyan">num_flights</font><font color = "silver">,</font>
<br/>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<font color = "#FF0080"><b>EXTRACT</b></font><font color = "cyan">(</font><font color = "cyan">DAY</font>&nbsp;<font color = "royalblue">FROM</font>&nbsp;<font color = "cyan">f</font><font color = "silver">.</font><font color = "cyan">ARRIVAL_TIME</font>&nbsp;<font color = "silver">::</font>&nbsp;<font color = "royalblue">DATE</font><font color = "cyan">)</font>&nbsp;<font color = "royalblue">AS</font>&nbsp;<font color = "cyan">day_flight</font>
<br/>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<font color = "royalblue">FROM</font>&nbsp;&nbsp;&nbsp;<font color = "cyan">flights</font>&nbsp;<font color = "cyan">f</font>
<br/>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<font color = "cyan">JOIN</font>&nbsp;<font color = "cyan">airports</font>&nbsp;<font color = "cyan">a</font>
<br/>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<font color = "royalblue">ON</font>&nbsp;<font color = "cyan">a</font><font color = "silver">.</font><font color = "cyan">AIRPORT_CODE</font>&nbsp;<font color = "silver">=</font>&nbsp;<font color = "cyan">f</font><font color = "silver">.</font><font color = "cyan">ARRIVAL_AIRPORT</font>
<br/>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<font color = "royalblue">WHERE</font>&nbsp;&nbsp;<font color = "cyan">f</font><font color = "silver">.</font><font color = "cyan">ARRIVAL_TIME</font>&nbsp;<font color = "silver">::</font>&nbsp;<font color = "royalblue">DATE</font>&nbsp;<font color = "silver">&lt;</font>&nbsp;<font color = "green">'2018-09-01'</font>
<br/>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<font color = "royalblue">AND</font>&nbsp;<font color = "cyan">f</font><font color = "silver">.</font><font color = "cyan">ARRIVAL_TIME</font>&nbsp;<font color = "silver">::</font>&nbsp;<font color = "royalblue">DATE</font>&nbsp;<font color = "silver">&gt;=</font>&nbsp;<font color = "green">'2018-08-01'</font>
<br/>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<font color = "royalblue">GROUP</font>&nbsp;&nbsp;<font color = "royalblue">BY</font>&nbsp;<font color = "cyan">a</font><font color = "silver">.</font><font color = "cyan">CITY</font><font color = "silver">,</font>
<br/>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<font color = "cyan">DAY_FLIGHT</font><font color = "cyan">)</font>
<br/><font color = "royalblue">SELECT</font>&nbsp;<font color = "royalblue">DISTINCT</font>&nbsp;<font color = "cyan">t</font><font color = "silver">.</font><font color = "cyan">CITY</font><font color = "silver">,</font>
<br/>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<font color = "#FF0080"><b>AVG</b></font>&nbsp;<font color = "cyan">(</font><font color = "cyan">t</font><font color = "silver">.</font><font color = "cyan">NUM_FLIGHTS</font><font color = "cyan">)</font>&nbsp;<font color = "royalblue">AS</font>&nbsp;<font color = "cyan">average_flights</font>
<br/><font color = "royalblue">FROM</font>&nbsp;&nbsp;&nbsp;<font color = "cyan">FLIGHTS_08</font>&nbsp;<font color = "cyan">t</font>
<br/><font color = "royalblue">GROUP</font>&nbsp;&nbsp;<font color = "royalblue">BY</font>&nbsp;<font color = "cyan">t</font><font color = "silver">.</font><font color = "cyan">CITY</font>&nbsp;
</font>


### 6 запрос


Установите фестивали, которые проходили **с 23 июля по 30 сентября 2018 года в Москве**, и номер недели, в которую они проходили. Выведите название фестиваля `festival_name` и номер недели `festival_week`.

<font face="Courier New" size="2">
<font color = "royalblue">SELECT</font>&nbsp;<font color = "cyan">f</font><font color = "silver">.</font><font color = "cyan">FESTIVAL_NAME</font>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<font color = "royalblue">AS</font>&nbsp;<font color = "cyan">festival_name</font><font color = "silver">,</font>
<br/>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<font color = "#FF0080"><b>EXTRACT</b></font><font color = "cyan">(</font><font color = "cyan">WEEK</font>&nbsp;<font color = "royalblue">FROM</font>&nbsp;<font color = "cyan">f</font><font color = "silver">.</font><font color = "cyan">FESTIVAL_DATE</font><font color = "cyan">)</font>&nbsp;<font color = "royalblue">AS</font>&nbsp;<font color = "cyan">festival_week</font>
<br/><font color = "royalblue">FROM</font>&nbsp;&nbsp;&nbsp;<font color = "cyan">festivals</font>&nbsp;<font color = "cyan">f</font>
<br/><font color = "royalblue">WHERE</font>&nbsp;&nbsp;<font color = "cyan">f</font><font color = "silver">.</font><font color = "cyan">FESTIVAL_CITY</font>&nbsp;<font color = "royalblue">LIKE</font>&nbsp;<font color = "green">'%Москва%'</font>
<br/>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<font color = "royalblue">AND</font>&nbsp;<font color = "cyan">f</font><font color = "silver">.</font><font color = "cyan">FESTIVAL_DATE</font>&nbsp;<font color = "silver">&lt;</font>&nbsp;<font color = "green">'2018-09-30'</font>
<br/>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<font color = "royalblue">AND</font>&nbsp;<font color = "cyan">f</font><font color = "silver">.</font><font color = "cyan">FESTIVAL_DATE</font>&nbsp;<font color = "silver">&gt;</font>&nbsp;<font color = "green">'2018-07-23'</font>&nbsp;
</font>


### 7 запрос



Для каждой недели с 23 июля по 30 сентября 2018 года посчитайте билеты, которые купили на рейсы в Москву (номер недели `week_number` и количество билетов `ticket_amount`). 

<font face="Courier New" size="2">
<font color = "royalblue">WITH</font>&nbsp;<font color = "cyan">TICKETS_AMOUNT</font>
<br/>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<font color = "royalblue">AS</font>&nbsp;<font color = "cyan">(</font><font color = "royalblue">SELECT</font>&nbsp;<font color = "#FF0080"><b>EXTRACT</b></font>&nbsp;<font color = "cyan">(</font><font color = "cyan">WEEK</font>&nbsp;<font color = "royalblue">FROM</font>&nbsp;<font color = "fuchsia"><i>Cast</i></font><font color = "cyan">(</font><font color = "cyan">f</font><font color = "silver">.</font><font color = "cyan">ARRIVAL_TIME</font>&nbsp;<font color = "royalblue">AS</font>&nbsp;<font color = "black"><i>DATE</i></font><font color = "cyan">)</font><font color = "cyan">)</font>&nbsp;<font color = "royalblue">AS</font>&nbsp;<font color = "cyan">week_number</font><font color = "silver">,</font>
<br/>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<font color = "fuchsia"><i>Count</i></font>&nbsp;<font color = "cyan">(</font><font color = "cyan">tf</font><font color = "silver">.</font><font color = "cyan">TICKET_NO</font><font color = "cyan">)</font>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<font color = "royalblue">AS</font>
<br/>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<font color = "cyan">ticket_amount</font>
<br/>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<font color = "royalblue">FROM</font>&nbsp;&nbsp;&nbsp;<font color = "cyan">flights</font>&nbsp;<font color = "cyan">f</font>
<br/>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<font color = "royalblue">JOIN</font>&nbsp;<font color = "cyan">airports</font>&nbsp;<font color = "cyan">a</font>
<br/>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<font color = "royalblue">ON</font>&nbsp;<font color = "cyan">a</font><font color = "silver">.</font><font color = "cyan">AIRPORT_CODE</font>&nbsp;<font color = "silver">=</font>&nbsp;<font color = "cyan">f</font><font color = "silver">.</font><font color = "cyan">ARRIVAL_AIRPORT</font>
<br/>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<font color = "royalblue">JOIN</font>&nbsp;<font color = "cyan">ticket_flights</font>&nbsp;<font color = "cyan">tf</font>
<br/>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<font color = "royalblue">ON</font>&nbsp;<font color = "cyan">tf</font><font color = "silver">.</font><font color = "cyan">FLIGHT_ID</font>&nbsp;<font color = "silver">=</font>&nbsp;<font color = "cyan">f</font><font color = "silver">.</font><font color = "cyan">FLIGHT_ID</font>
<br/>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<font color = "royalblue">WHERE</font>&nbsp;&nbsp;<font color = "cyan">a</font><font color = "silver">.</font><font color = "cyan">CITY</font>&nbsp;<font color = "silver">=</font>&nbsp;<font color = "green">'Москва'</font>
<br/>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<font color = "royalblue">AND</font>&nbsp;<font color = "fuchsia"><i>Cast</i></font><font color = "cyan">(</font><font color = "cyan">f</font><font color = "silver">.</font><font color = "cyan">ARRIVAL_TIME</font>&nbsp;<font color = "royalblue">AS</font>&nbsp;<font color = "black"><i>DATE</i></font><font color = "cyan">)</font>&nbsp;<font color = "royalblue">BETWEEN</font>
<br/>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<font color = "green">'2018-07-23'</font>&nbsp;<font color = "royalblue">AND</font>&nbsp;<font color = "green">'2018-09-30'</font>
<br/>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<font color = "royalblue">GROUP</font>&nbsp;&nbsp;<font color = "royalblue">BY</font>&nbsp;<font color = "cyan">WEEK_NUMBER</font><font color = "cyan">)</font><font color = "silver">,</font>
<br/>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<font color = "cyan">FESTIVAL_TICKETS</font>
<br/>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<font color = "royalblue">AS</font>&nbsp;<font color = "cyan">(</font><font color = "royalblue">SELECT</font>&nbsp;<font color = "cyan">f</font><font color = "silver">.</font><font color = "cyan">FESTIVAL_NAME</font>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<font color = "royalblue">AS</font>&nbsp;<font color = "cyan">festival_name</font><font color = "silver">,</font>
<br/>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<font color = "#FF0080"><b>EXTRACT</b></font><font color = "cyan">(</font><font color = "cyan">WEEK</font>&nbsp;<font color = "royalblue">FROM</font>&nbsp;<font color = "cyan">f</font><font color = "silver">.</font><font color = "cyan">FESTIVAL_DATE</font><font color = "cyan">)</font>&nbsp;<font color = "royalblue">AS</font>&nbsp;<font color = "cyan">festival_week</font>
<br/>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<font color = "royalblue">FROM</font>&nbsp;&nbsp;&nbsp;<font color = "cyan">festivals</font>&nbsp;<font color = "cyan">f</font>
<br/>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<font color = "royalblue">WHERE</font>&nbsp;&nbsp;<font color = "cyan">f</font><font color = "silver">.</font><font color = "cyan">FESTIVAL_CITY</font>&nbsp;<font color = "royalblue">LIKE</font>&nbsp;<font color = "green">'%Москва%'</font>
<br/>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<font color = "royalblue">AND</font>&nbsp;<font color = "fuchsia"><i>Cast</i></font><font color = "cyan">(</font><font color = "cyan">f</font><font color = "silver">.</font><font color = "cyan">FESTIVAL_DATE</font>&nbsp;<font color = "royalblue">AS</font>&nbsp;<font color = "black"><i>DATE</i></font><font color = "cyan">)</font>&nbsp;<font color = "royalblue">BETWEEN</font>
<br/>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<font color = "green">'2018-07-23'</font>&nbsp;<font color = "royalblue">AND</font>&nbsp;<font color = "green">'2018-09-30'</font><font color = "cyan">)</font>
<br/><font color = "royalblue">SELECT</font>&nbsp;<font color = "cyan">ta</font><font color = "silver">.</font><font color = "cyan">WEEK_NUMBER</font><font color = "silver">,</font>
<br/>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<font color = "cyan">ta</font><font color = "silver">.</font><font color = "cyan">TICKET_AMOUNT</font><font color = "silver">,</font>
<br/>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<font color = "cyan">ft</font><font color = "silver">.</font><font color = "cyan">FESTIVAL_WEEK</font><font color = "silver">,</font>
<br/>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<font color = "cyan">ft</font><font color = "silver">.</font><font color = "cyan">FESTIVAL_NAME</font>
<br/><font color = "royalblue">FROM</font>&nbsp;&nbsp;&nbsp;<font color = "cyan">TICKETS_AMOUNT</font>&nbsp;<font color = "cyan">ta</font>
<br/>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<font color = "royalblue">LEFT</font>&nbsp;<font color = "royalblue">JOIN</font>&nbsp;<font color = "cyan">FESTIVAL_TICKETS</font>&nbsp;<font color = "cyan">ft</font>
<br/>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<font color = "royalblue">ON</font>&nbsp;<font color = "cyan">ta</font><font color = "silver">.</font><font color = "cyan">WEEK_NUMBER</font>&nbsp;<font color = "silver">=</font>&nbsp;<font color = "cyan">ft</font><font color = "silver">.</font><font color = "cyan">FESTIVAL_WEEK</font>&nbsp;
</font>
